In [1]:
import numpy as np
import os
import datetime
import requests
import shutil
import zipfile
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from scripts.reconstruction_minimal import createAudio
import scipy.io.wavfile as wavfile
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Data prepare

In [2]:
#Prepare data for CNN network
def prepare_cnn(data, spectrogram, window_size):
    input = np.zeros((data.shape[0] - window_size, window_size, data.shape[1]))
    output = np.zeros((spectrogram.shape[0] - window_size, spectrogram.shape[1]))
    for i in range(data.shape[0] - window_size):
      output[i, :] = spectrogram[i + window_size, :]
      input[i, :, :] = data[i:i+window_size, :]
        
    return (input, output)

In [3]:
#Prepare dataset for CNN network
def prepare_cnn_dataset(X_train, Y_train, X_test, Y_test, window_size):
    for i in range(len(X_train)):
        (X_train[i], Y_train[i]) = prepare_cnn(X_train[i], Y_train[i], window_size)
        (X_test[i], Y_test[i]) = prepare_cnn(X_test[i], Y_test[i], window_size)

In [4]:
# Read the dataset and prepare for training
def generate_datasets(pt, feat_path=r'./features'):
    
    r"""
    Generate train and test datasets from the one sample of the raw data.
    param pt: name of the sample
    param feat_path: path of the raw data file
  """
    
    spectrogram = np.load(os.path.join(feat_path,f'{pt}_spec.npy'))
    data = np.load(os.path.join(feat_path,f'{pt}_feat.npy'))

    # Generate train, validation and test datasets
    X_train, X_test, Y_train, Y_test = train_test_split(data, spectrogram, test_size=0.1, shuffle=False)

    # Shuffle train data
    randomize = np.arange(len(X_train))
    np.random.shuffle(randomize)
    X_train = X_train[randomize]
    Y_train = Y_train[randomize]
    
    # Stnadardize input
    mu_input = np.mean(X_train, axis=0)
    std_input = np.std(X_train, axis=0)
    X_train = (X_train-mu_input)/std_input
    X_test = (X_test-mu_input)/std_input

    # Reduce Dimensions
    pca = PCA()
    pca.fit(X_train)
    X_train = np.dot(X_train, pca.components_[:50,:].T)
    X_test = np.dot(X_test, pca.components_[:50,:].T)

    # Standardize output
    mu_output = np.mean(Y_train, axis=0)
    std_output = np.std(Y_train, axis=0)
    Y_train = (Y_train-mu_output)/std_output
    Y_test = (Y_test-mu_output)/std_output


    return (X_train, Y_train, X_test, Y_test, mu_input, std_input, mu_output, std_output)

In [5]:
# Read all dataset and prepare for training
def generate_train_test_datasets():
    norm_params = {}
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    pts = ['sub-%02d'%i for i in range(1, 11)]
    for i, pt in enumerate(pts):
        (x_train, y_train, x_test, y_test, mu_input, std_input, mu_output, std_output) = generate_datasets(pt)
        norm_params[str(i)]={'mu_input':mu_input, 'std_input':std_input, 'mu_output':mu_output, 'std_output':std_output}
        X_train.append(x_train)
        Y_train.append(y_train)
        X_test.append(x_test)
        Y_test.append(y_test)
    return (X_train, Y_train, X_test, Y_test, norm_params)

# Modelling

In [6]:
# Synthesize audio files
def synthesize_audio(pred, spectogram, result_path, result_name):
    winLength = 0.05
    frameshift = 0.01
    audiosr = 16000
    if not os.path.isdir(result_path):
        os.makedirs(result_path)
    
    #Synthesize waveform from spectrogram using Griffin-Lim
    reconstructedWav = createAudio(pred,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
    wavfile.write(os.path.join(result_path,f'{result_name}_predicted.wav'),int(audiosr),reconstructedWav)

    #For comparison synthesize the original spectrogram with Griffin-Lim
    origWav = createAudio(spectogram,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
    wavfile.write(os.path.join(result_path,f'{result_name}_orig_synthesized.wav'),int(audiosr),origWav)

In [7]:
# Create FC-DNN neural network
def create_fndnn(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(units=60, input_dim=input_dim, activation='relu'))
    model.add(Dense(units=30, activation='relu'))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dense(units=output_dim, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [8]:
# Create CNN neural network
def make_1d_convnet(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=[40, 40, 40]):

    model = Sequential()
    model.add(Conv1D(filters=nb_filter[0], kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)))
    model.add(Conv1D(filters=nb_filter[1], kernel_size=filter_length, activation='relu'))
    model.add(Conv1D(filters=nb_filter[2], kernel_size=filter_length, activation='relu'))
    model.add(Flatten())
    model.add(Dense(units=30, activation='relu'))
    model.add(Dense(nb_outputs, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [9]:
# Train models, evaluate the results and generate syntetised audio files
def train_test_models(model, X_train, Y_train, X_test, Y_test, result_path, batch_size = 512):
    epochs = 1000

    for i in range(len(X_test)):
        log_folder = f"logs/sample{i+1}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        early_stopping=EarlyStopping(patience=50, verbose=1)
        checkpointer=ModelCheckpoint(filepath='fcdnn.hdf5', save_best_only=True, verbose=1)
        tensorboard = TensorBoard(log_dir=log_folder, histogram_freq=1, write_graph=True, write_images=True, update_freq='epoch', profile_batch=2, embeddings_freq=1)
        
        print(f"Training for sample: {i+1}")
        model.fit(X_train[i], Y_train[i], validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, early_stopping, tensorboard])
        y_pred = model.predict(X_test[i])
        mae = mean_absolute_error(Y_test[i], y_pred)
        print(f'Mean absolute error of test set:{mae}\n\n')
        
        pred = y_pred * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output']
        spectogram = Y_test[i] * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output']
        synthesize_audio(pred, spectogram, result_path=result_path, result_name=f"sub-{i+1}")

# Train separate models

We created 3 different fc-dnn neural network and a cnn network for find the best architechture.

In [11]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_train_test_datasets()
model = create_fndnn(input_dim=X_train[0].shape[1], output_dim=Y_train[0].shape[1])
train_test_models(model, X_train, Y_train, X_test, Y_test, result_path="audio/dnn_results_0")
# The original architechture was the best.

Training for sample: 1
Epoch 1/1000

Epoch 1: val_loss improved from inf to 1.06176, saving model to fcdnn.hdf5
48/48 - 2s - loss: 1.6677 - mae: 0.9413 - val_loss: 1.0618 - val_mae: 0.7835 - 2s/epoch - 36ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 1.06176 to 1.01552, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0197 - mae: 0.7754 - val_loss: 1.0155 - val_mae: 0.7721 - 246ms/epoch - 5ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 1.01552 to 1.00833, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0013 - mae: 0.7703 - val_loss: 1.0083 - val_mae: 0.7698 - 299ms/epoch - 6ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 1.00833 to 1.00050, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9955 - mae: 0.7690 - val_loss: 1.0005 - val_mae: 0.7681 - 308ms/epoch - 6ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 1.00050 to 0.96994, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9793 - mae: 0.7646 - val_loss: 0.9699 - val_mae: 0.7602 - 284ms/epoch - 6ms/step
Epoch 6/

Epoch 44/1000

Epoch 44: val_loss improved from 0.41641 to 0.41542, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3232 - mae: 0.3808 - val_loss: 0.4154 - val_mae: 0.4290 - 252ms/epoch - 5ms/step
Epoch 45/1000

Epoch 45: val_loss improved from 0.41542 to 0.41112, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3223 - mae: 0.3794 - val_loss: 0.4111 - val_mae: 0.4278 - 268ms/epoch - 6ms/step
Epoch 46/1000

Epoch 46: val_loss did not improve from 0.41112
48/48 - 0s - loss: 0.3198 - mae: 0.3785 - val_loss: 0.4129 - val_mae: 0.4253 - 283ms/epoch - 6ms/step
Epoch 47/1000

Epoch 47: val_loss improved from 0.41112 to 0.40712, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3179 - mae: 0.3766 - val_loss: 0.4071 - val_mae: 0.4273 - 252ms/epoch - 5ms/step
Epoch 48/1000

Epoch 48: val_loss did not improve from 0.40712
48/48 - 0s - loss: 0.3151 - mae: 0.3749 - val_loss: 0.4092 - val_mae: 0.4273 - 215ms/epoch - 4ms/step
Epoch 49/1000

Epoch 49: val_loss improved from 0.40712 to 0.40400, saving mode

Epoch 90/1000

Epoch 90: val_loss improved from 0.36791 to 0.36705, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.2616 - mae: 0.3401 - val_loss: 0.3671 - val_mae: 0.3985 - 221ms/epoch - 5ms/step
Epoch 91/1000

Epoch 91: val_loss improved from 0.36705 to 0.36246, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.2598 - mae: 0.3383 - val_loss: 0.3625 - val_mae: 0.3958 - 251ms/epoch - 5ms/step
Epoch 92/1000

Epoch 92: val_loss did not improve from 0.36246
48/48 - 0s - loss: 0.2601 - mae: 0.3387 - val_loss: 0.3672 - val_mae: 0.4014 - 267ms/epoch - 6ms/step
Epoch 93/1000

Epoch 93: val_loss improved from 0.36246 to 0.36149, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.2592 - mae: 0.3379 - val_loss: 0.3615 - val_mae: 0.3955 - 268ms/epoch - 6ms/step
Epoch 94/1000

Epoch 94: val_loss did not improve from 0.36149
48/48 - 0s - loss: 0.2588 - mae: 0.3376 - val_loss: 0.3637 - val_mae: 0.3990 - 302ms/epoch - 6ms/step
Epoch 95/1000

Epoch 95: val_loss improved from 0.36149 to 0.35830, saving mode

48/48 - 0s - loss: 0.2263 - mae: 0.3185 - val_loss: 0.3478 - val_mae: 0.3841 - 252ms/epoch - 5ms/step
Epoch 137/1000

Epoch 137: val_loss did not improve from 0.34468
48/48 - 0s - loss: 0.2256 - mae: 0.3175 - val_loss: 0.3492 - val_mae: 0.3858 - 212ms/epoch - 4ms/step
Epoch 138/1000

Epoch 138: val_loss improved from 0.34468 to 0.34241, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.2251 - mae: 0.3177 - val_loss: 0.3424 - val_mae: 0.3837 - 283ms/epoch - 6ms/step
Epoch 139/1000

Epoch 139: val_loss did not improve from 0.34241
48/48 - 0s - loss: 0.2239 - mae: 0.3167 - val_loss: 0.3443 - val_mae: 0.3840 - 283ms/epoch - 6ms/step
Epoch 140/1000

Epoch 140: val_loss did not improve from 0.34241
48/48 - 0s - loss: 0.2230 - mae: 0.3159 - val_loss: 0.3446 - val_mae: 0.3859 - 252ms/epoch - 5ms/step
Epoch 141/1000

Epoch 141: val_loss did not improve from 0.34241
48/48 - 0s - loss: 0.2240 - mae: 0.3172 - val_loss: 0.3465 - val_mae: 0.3853 - 227ms/epoch - 5ms/step
Epoch 142/1000

Epoch 142: val_

48/48 - 0s - loss: 0.2059 - mae: 0.3055 - val_loss: 0.3294 - val_mae: 0.3766 - 231ms/epoch - 5ms/step
Epoch 184/1000

Epoch 184: val_loss improved from 0.32454 to 0.32072, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.2061 - mae: 0.3056 - val_loss: 0.3207 - val_mae: 0.3736 - 284ms/epoch - 6ms/step
Epoch 185/1000

Epoch 185: val_loss did not improve from 0.32072
48/48 - 0s - loss: 0.2050 - mae: 0.3051 - val_loss: 0.3243 - val_mae: 0.3764 - 252ms/epoch - 5ms/step
Epoch 186/1000

Epoch 186: val_loss did not improve from 0.32072
48/48 - 0s - loss: 0.2050 - mae: 0.3050 - val_loss: 0.3242 - val_mae: 0.3719 - 238ms/epoch - 5ms/step
Epoch 187/1000

Epoch 187: val_loss did not improve from 0.32072
48/48 - 0s - loss: 0.2035 - mae: 0.3035 - val_loss: 0.3213 - val_mae: 0.3762 - 221ms/epoch - 5ms/step
Epoch 188/1000

Epoch 188: val_loss did not improve from 0.32072
48/48 - 0s - loss: 0.2042 - mae: 0.3049 - val_loss: 0.3250 - val_mae: 0.3739 - 239ms/epoch - 5ms/step
Epoch 189/1000

Epoch 189: val_

...


Epoch 103: val_loss did not improve from 0.35575
46/46 - 0s - loss: 0.3069 - mae: 0.3446 - val_loss: 0.3565 - val_mae: 0.3667 - 237ms/epoch - 5ms/step
Epoch 104/1000

Epoch 104: val_loss did not improve from 0.35575
46/46 - 0s - loss: 0.3062 - mae: 0.3443 - val_loss: 0.3570 - val_mae: 0.3659 - 268ms/epoch - 6ms/step
Epoch 105/1000

Epoch 105: val_loss did not improve from 0.35575
46/46 - 0s - loss: 0.3054 - mae: 0.3440 - val_loss: 0.3574 - val_mae: 0.3721 - 249ms/epoch - 5ms/step
Epoch 106/1000

Epoch 106: val_loss did not improve from 0.35575
46/46 - 0s - loss: 0.3048 - mae: 0.3446 - val_loss: 0.3579 - val_mae: 0.3654 - 224ms/epoch - 5ms/step
Epoch 107/1000

Epoch 107: val_loss improved from 0.35575 to 0.35448, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.3063 - mae: 0.3450 - val_loss: 0.3545 - val_mae: 0.3656 - 286ms/epoch - 6ms/step
Epoch 108/1000

Epoch 108: val_loss did not improve from 0.35448
46/46 - 0s - loss: 0.3039 - mae: 0.3434 - val_loss: 0.3549 - val_mae: 0.3654 - 236m

Epoch 150/1000

Epoch 150: val_loss did not improve from 0.33052
46/46 - 0s - loss: 0.2821 - mae: 0.3324 - val_loss: 0.3320 - val_mae: 0.3558 - 236ms/epoch - 5ms/step
Epoch 151/1000

Epoch 151: val_loss did not improve from 0.33052
46/46 - 0s - loss: 0.2810 - mae: 0.3320 - val_loss: 0.3351 - val_mae: 0.3553 - 253ms/epoch - 5ms/step
Epoch 152/1000

Epoch 152: val_loss did not improve from 0.33052
46/46 - 0s - loss: 0.2816 - mae: 0.3321 - val_loss: 0.3330 - val_mae: 0.3534 - 264ms/epoch - 6ms/step
Epoch 153/1000

Epoch 153: val_loss improved from 0.33052 to 0.33052, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2803 - mae: 0.3313 - val_loss: 0.3305 - val_mae: 0.3564 - 293ms/epoch - 6ms/step
Epoch 154/1000

Epoch 154: val_loss improved from 0.33052 to 0.32612, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2787 - mae: 0.3308 - val_loss: 0.3261 - val_mae: 0.3515 - 260ms/epoch - 6ms/step
Epoch 155/1000

Epoch 155: val_loss did not improve from 0.32612
46/46 - 0s - loss: 0.2785 - mae: 0.3

Epoch 196/1000

Epoch 196: val_loss improved from 0.31239 to 0.30901, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2649 - mae: 0.3242 - val_loss: 0.3090 - val_mae: 0.3432 - 315ms/epoch - 7ms/step
Epoch 197/1000

Epoch 197: val_loss did not improve from 0.30901
46/46 - 0s - loss: 0.2629 - mae: 0.3228 - val_loss: 0.3119 - val_mae: 0.3461 - 221ms/epoch - 5ms/step
Epoch 198/1000

Epoch 198: val_loss did not improve from 0.30901
46/46 - 0s - loss: 0.2647 - mae: 0.3245 - val_loss: 0.3169 - val_mae: 0.3433 - 236ms/epoch - 5ms/step
Epoch 199/1000

Epoch 199: val_loss did not improve from 0.30901
46/46 - 0s - loss: 0.2630 - mae: 0.3231 - val_loss: 0.3169 - val_mae: 0.3483 - 285ms/epoch - 6ms/step
Epoch 200/1000

Epoch 200: val_loss did not improve from 0.30901
46/46 - 0s - loss: 0.2638 - mae: 0.3250 - val_loss: 0.3153 - val_mae: 0.3467 - 267ms/epoch - 6ms/step
Epoch 201/1000

Epoch 201: val_loss did not improve from 0.30901
46/46 - 0s - loss: 0.2625 - mae: 0.3230 - val_loss: 0.3156 - val_mae

Epoch 244/1000

Epoch 244: val_loss did not improve from 0.30328
46/46 - 0s - loss: 0.2475 - mae: 0.3156 - val_loss: 0.3056 - val_mae: 0.3382 - 243ms/epoch - 5ms/step
Epoch 245/1000

Epoch 245: val_loss improved from 0.30328 to 0.30191, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2491 - mae: 0.3165 - val_loss: 0.3019 - val_mae: 0.3377 - 266ms/epoch - 6ms/step
Epoch 246/1000

Epoch 246: val_loss did not improve from 0.30191
46/46 - 0s - loss: 0.2470 - mae: 0.3151 - val_loss: 0.3034 - val_mae: 0.3384 - 224ms/epoch - 5ms/step
Epoch 247/1000

Epoch 247: val_loss did not improve from 0.30191
46/46 - 0s - loss: 0.2472 - mae: 0.3155 - val_loss: 0.3031 - val_mae: 0.3367 - 250ms/epoch - 5ms/step
Epoch 248/1000

Epoch 248: val_loss did not improve from 0.30191
46/46 - 0s - loss: 0.2468 - mae: 0.3153 - val_loss: 0.3024 - val_mae: 0.3387 - 236ms/epoch - 5ms/step
Epoch 249/1000

Epoch 249: val_loss did not improve from 0.30191
46/46 - 0s - loss: 0.2457 - mae: 0.3146 - val_loss: 0.3028 - val_mae

46/46 - 0s - loss: 0.2378 - mae: 0.3106 - val_loss: 0.3038 - val_mae: 0.3369 - 252ms/epoch - 5ms/step
Epoch 293/1000

Epoch 293: val_loss did not improve from 0.29790
46/46 - 0s - loss: 0.2364 - mae: 0.3094 - val_loss: 0.3038 - val_mae: 0.3390 - 237ms/epoch - 5ms/step
Epoch 294/1000

Epoch 294: val_loss did not improve from 0.29790
46/46 - 0s - loss: 0.2368 - mae: 0.3099 - val_loss: 0.3023 - val_mae: 0.3350 - 253ms/epoch - 5ms/step
Epoch 295/1000

Epoch 295: val_loss did not improve from 0.29790
46/46 - 0s - loss: 0.2399 - mae: 0.3118 - val_loss: 0.3046 - val_mae: 0.3410 - 253ms/epoch - 5ms/step
Epoch 296/1000

Epoch 296: val_loss did not improve from 0.29790
46/46 - 0s - loss: 0.2397 - mae: 0.3127 - val_loss: 0.3018 - val_mae: 0.3362 - 253ms/epoch - 5ms/step
Epoch 297/1000

Epoch 297: val_loss did not improve from 0.29790
46/46 - 0s - loss: 0.2384 - mae: 0.3108 - val_loss: 0.3023 - val_mae: 0.3356 - 221ms/epoch - 5ms/step
Epoch 298/1000

Epoch 298: val_loss did not improve from 0.2979

In [12]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_train_test_datasets()
model = make_1d_convnet(window_size=4, filter_length=2, nb_filter=[40, 40, 40], nb_input_series=50, nb_outputs=23)
prepare_cnn_dataset(X_train, Y_train, X_test, Y_test, window_size=4)
train_test_models(model, X_train, Y_train, X_test, Y_test, result_path="audio/cnn_results")

Training for sample: 1
Epoch 1/1000

Epoch 1: val_loss improved from inf to 1.00681, saving model to fcdnn.hdf5
48/48 - 1s - loss: 1.1146 - mae: 0.8080 - val_loss: 1.0068 - val_mae: 0.7756 - 1s/epoch - 26ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 1.00681 to 1.00290, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0025 - mae: 0.7699 - val_loss: 1.0029 - val_mae: 0.7751 - 425ms/epoch - 9ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 1.00290 to 1.00224, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0004 - mae: 0.7688 - val_loss: 1.0022 - val_mae: 0.7741 - 363ms/epoch - 8ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 1.00224 to 1.00204, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9999 - mae: 0.7681 - val_loss: 1.0020 - val_mae: 0.7740 - 380ms/epoch - 8ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 1.00204 to 1.00200, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9996 - mae: 0.7688 - val_loss: 1.0020 - val_mae: 0.7746 - 441ms/epoch - 9ms/step
Epoch 6/

Epoch 50/1000

Epoch 50: val_loss did not improve from 1.00200
48/48 - 0s - loss: 0.5594 - mae: 0.5276 - val_loss: 1.4532 - val_mae: 0.8664 - 347ms/epoch - 7ms/step
Epoch 51/1000

Epoch 51: val_loss did not improve from 1.00200
48/48 - 0s - loss: 0.5563 - mae: 0.5258 - val_loss: 1.4591 - val_mae: 0.8697 - 372ms/epoch - 8ms/step
Epoch 52/1000

Epoch 52: val_loss did not improve from 1.00200
48/48 - 0s - loss: 0.5536 - mae: 0.5253 - val_loss: 1.5207 - val_mae: 0.8799 - 381ms/epoch - 8ms/step
Epoch 53/1000

Epoch 53: val_loss did not improve from 1.00200
48/48 - 0s - loss: 0.5541 - mae: 0.5253 - val_loss: 1.5127 - val_mae: 0.8786 - 387ms/epoch - 8ms/step
Epoch 54/1000

Epoch 54: val_loss did not improve from 1.00200
48/48 - 0s - loss: 0.5504 - mae: 0.5234 - val_loss: 1.4821 - val_mae: 0.8691 - 407ms/epoch - 8ms/step
Epoch 55/1000

Epoch 55: val_loss did not improve from 1.00200
48/48 - 0s - loss: 0.5424 - mae: 0.5186 - val_loss: 1.5313 - val_mae: 0.8761 - 375ms/epoch - 8ms/step
Epoch 55: 


Epoch 43: val_loss did not improve from 0.98377
48/48 - 0s - loss: 0.7924 - mae: 0.6741 - val_loss: 1.1803 - val_mae: 0.8517 - 372ms/epoch - 8ms/step
Epoch 44/1000

Epoch 44: val_loss did not improve from 0.98377
48/48 - 0s - loss: 0.7859 - mae: 0.6709 - val_loss: 1.1877 - val_mae: 0.8583 - 394ms/epoch - 8ms/step
Epoch 45/1000

Epoch 45: val_loss did not improve from 0.98377
48/48 - 0s - loss: 0.7753 - mae: 0.6651 - val_loss: 1.2060 - val_mae: 0.8657 - 380ms/epoch - 8ms/step
Epoch 46/1000

Epoch 46: val_loss did not improve from 0.98377
48/48 - 0s - loss: 0.7699 - mae: 0.6631 - val_loss: 1.2086 - val_mae: 0.8641 - 347ms/epoch - 7ms/step
Epoch 47/1000

Epoch 47: val_loss did not improve from 0.98377
48/48 - 0s - loss: 0.7621 - mae: 0.6598 - val_loss: 1.2315 - val_mae: 0.8671 - 363ms/epoch - 8ms/step
Epoch 48/1000

Epoch 48: val_loss did not improve from 0.98377
48/48 - 0s - loss: 0.7521 - mae: 0.6549 - val_loss: 1.2412 - val_mae: 0.8745 - 363ms/epoch - 8ms/step
Epoch 49/1000

Epoch 49:

48/48 - 0s - loss: 0.6221 - mae: 0.6129 - val_loss: 1.3692 - val_mae: 0.9239 - 396ms/epoch - 8ms/step
Epoch 38/1000

Epoch 38: val_loss did not improve from 1.03797
48/48 - 0s - loss: 0.6190 - mae: 0.6111 - val_loss: 1.3859 - val_mae: 0.9269 - 373ms/epoch - 8ms/step
Epoch 39/1000

Epoch 39: val_loss did not improve from 1.03797
48/48 - 0s - loss: 0.6104 - mae: 0.6058 - val_loss: 1.3954 - val_mae: 0.9288 - 397ms/epoch - 8ms/step
Epoch 40/1000

Epoch 40: val_loss did not improve from 1.03797
48/48 - 0s - loss: 0.6030 - mae: 0.6020 - val_loss: 1.3996 - val_mae: 0.9362 - 366ms/epoch - 8ms/step
Epoch 41/1000

Epoch 41: val_loss did not improve from 1.03797
48/48 - 0s - loss: 0.5988 - mae: 0.6001 - val_loss: 1.3964 - val_mae: 0.9400 - 394ms/epoch - 8ms/step
Epoch 42/1000

Epoch 42: val_loss did not improve from 1.03797
48/48 - 0s - loss: 0.5930 - mae: 0.5972 - val_loss: 1.4060 - val_mae: 0.9315 - 364ms/epoch - 8ms/step
Epoch 43/1000

Epoch 43: val_loss did not improve from 1.03797
48/48 - 0s

Epoch 33/1000

Epoch 33: val_loss did not improve from 0.99561
48/48 - 0s - loss: 0.7752 - mae: 0.6738 - val_loss: 1.1564 - val_mae: 0.8352 - 333ms/epoch - 7ms/step
Epoch 34/1000

Epoch 34: val_loss did not improve from 0.99561
48/48 - 0s - loss: 0.7673 - mae: 0.6662 - val_loss: 1.1687 - val_mae: 0.8366 - 344ms/epoch - 7ms/step
Epoch 35/1000

Epoch 35: val_loss did not improve from 0.99561
48/48 - 0s - loss: 0.7605 - mae: 0.6648 - val_loss: 1.1792 - val_mae: 0.8350 - 347ms/epoch - 7ms/step
Epoch 36/1000

Epoch 36: val_loss did not improve from 0.99561
48/48 - 0s - loss: 0.7546 - mae: 0.6592 - val_loss: 1.1833 - val_mae: 0.8486 - 347ms/epoch - 7ms/step
Epoch 37/1000

Epoch 37: val_loss did not improve from 0.99561
48/48 - 0s - loss: 0.7517 - mae: 0.6603 - val_loss: 1.1913 - val_mae: 0.8460 - 362ms/epoch - 8ms/step
Epoch 38/1000

Epoch 38: val_loss did not improve from 0.99561
48/48 - 0s - loss: 0.7429 - mae: 0.6557 - val_loss: 1.2005 - val_mae: 0.8506 - 382ms/epoch - 8ms/step
Epoch 39/1

...

Epoch 17/1000

Epoch 17: val_loss did not improve from 1.06166
48/48 - 0s - loss: 0.9339 - mae: 0.7129 - val_loss: 1.0945 - val_mae: 0.7622 - 363ms/epoch - 8ms/step
Epoch 18/1000

Epoch 18: val_loss did not improve from 1.06166
48/48 - 0s - loss: 0.9313 - mae: 0.7120 - val_loss: 1.0850 - val_mae: 0.7790 - 362ms/epoch - 8ms/step
Epoch 19/1000

Epoch 19: val_loss did not improve from 1.06166
48/48 - 0s - loss: 0.9265 - mae: 0.7102 - val_loss: 1.0929 - val_mae: 0.7971 - 363ms/epoch - 8ms/step
Epoch 20/1000

Epoch 20: val_loss did not improve from 1.06166
48/48 - 0s - loss: 0.9211 - mae: 0.7073 - val_loss: 1.0973 - val_mae: 0.7749 - 363ms/epoch - 8ms/step
Epoch 21/1000

Epoch 21: val_loss did not improve from 1.06166
48/48 - 0s - loss: 0.9156 - mae: 0.7067 - val_loss: 1.1092 - val_mae: 0.7542 - 412ms/epoch - 9ms/step
Epoch 22/1000

Epoch 22: val_loss did not improve from 1.06166
48/48 - 0s - loss: 0.9152 - mae: 0.7044 - val_loss: 1.0997 - val_mae: 0.7681 - 363ms/epoch - 8ms/step
Epoch 23/1


Epoch 9: val_loss did not improve from 1.04057
48/48 - 0s - loss: 0.9435 - mae: 0.7333 - val_loss: 1.0465 - val_mae: 0.7767 - 409ms/epoch - 9ms/step
Epoch 10/1000

Epoch 10: val_loss did not improve from 1.04057
48/48 - 0s - loss: 0.9394 - mae: 0.7304 - val_loss: 1.0503 - val_mae: 0.7654 - 363ms/epoch - 8ms/step
Epoch 11/1000

Epoch 11: val_loss did not improve from 1.04057
48/48 - 0s - loss: 0.9321 - mae: 0.7266 - val_loss: 1.0512 - val_mae: 0.7735 - 379ms/epoch - 8ms/step
Epoch 12/1000

Epoch 12: val_loss did not improve from 1.04057
48/48 - 0s - loss: 0.9273 - mae: 0.7240 - val_loss: 1.0577 - val_mae: 0.7938 - 379ms/epoch - 8ms/step
Epoch 13/1000

Epoch 13: val_loss did not improve from 1.04057
48/48 - 0s - loss: 0.9235 - mae: 0.7212 - val_loss: 1.0557 - val_mae: 0.7882 - 340ms/epoch - 7ms/step
Epoch 14/1000

Epoch 14: val_loss did not improve from 1.04057
48/48 - 0s - loss: 0.9174 - mae: 0.7190 - val_loss: 1.0593 - val_mae: 0.7939 - 377ms/epoch - 8ms/step
Epoch 15/1000

Epoch 15: 

Epoch 3/1000

Epoch 3: val_loss improved from 1.10360 to 1.08682, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0039 - mae: 0.7448 - val_loss: 1.0868 - val_mae: 0.7648 - 403ms/epoch - 8ms/step
Epoch 4/1000

Epoch 4: val_loss did not improve from 1.08682
48/48 - 0s - loss: 0.9913 - mae: 0.7406 - val_loss: 1.1023 - val_mae: 0.7741 - 354ms/epoch - 7ms/step
Epoch 5/1000

Epoch 5: val_loss did not improve from 1.08682
48/48 - 0s - loss: 0.9869 - mae: 0.7379 - val_loss: 1.0889 - val_mae: 0.7668 - 395ms/epoch - 8ms/step
Epoch 6/1000

Epoch 6: val_loss did not improve from 1.08682
48/48 - 0s - loss: 0.9788 - mae: 0.7330 - val_loss: 1.0923 - val_mae: 0.8207 - 377ms/epoch - 8ms/step
Epoch 7/1000

Epoch 7: val_loss did not improve from 1.08682
48/48 - 0s - loss: 0.9757 - mae: 0.7357 - val_loss: 1.0914 - val_mae: 0.7453 - 378ms/epoch - 8ms/step
Epoch 8/1000

Epoch 8: val_loss did not improve from 1.08682
48/48 - 0s - loss: 0.9697 - mae: 0.7301 - val_loss: 1.0872 - val_mae: 0.7922 - 491ms/epoch -

Epoch 53/1000

Epoch 53: val_loss did not improve from 1.08682
48/48 - 0s - loss: 0.8694 - mae: 0.6818 - val_loss: 1.1680 - val_mae: 0.8211 - 378ms/epoch - 8ms/step
Epoch 53: early stopping
94/94 [==============================] - 0s 1ms/step
Mean absolute error of test set:0.7607658698879277


Training for sample: 10
Epoch 1/1000

Epoch 1: val_loss improved from inf to 1.02928, saving model to fcdnn.hdf5
46/46 - 1s - loss: 1.1040 - mae: 0.7029 - val_loss: 1.0293 - val_mae: 0.7193 - 520ms/epoch - 11ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 1.02928 to 1.01540, saving model to fcdnn.hdf5
46/46 - 0s - loss: 1.0365 - mae: 0.6795 - val_loss: 1.0154 - val_mae: 0.6277 - 410ms/epoch - 9ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 1.01540 to 1.00770, saving model to fcdnn.hdf5
46/46 - 0s - loss: 1.0098 - mae: 0.6559 - val_loss: 1.0077 - val_mae: 0.7065 - 394ms/epoch - 9ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 1.00770 to 0.99492, saving model to fcdnn.hdf5
46/46


Epoch 48: val_loss did not improve from 0.99492
46/46 - 0s - loss: 0.9042 - mae: 0.6183 - val_loss: 1.0566 - val_mae: 0.6643 - 362ms/epoch - 8ms/step
Epoch 49/1000

Epoch 49: val_loss did not improve from 0.99492
46/46 - 0s - loss: 0.9023 - mae: 0.6159 - val_loss: 1.0626 - val_mae: 0.7067 - 322ms/epoch - 7ms/step
Epoch 50/1000

Epoch 50: val_loss did not improve from 0.99492
46/46 - 0s - loss: 0.9013 - mae: 0.6236 - val_loss: 1.0615 - val_mae: 0.6849 - 347ms/epoch - 8ms/step
Epoch 51/1000

Epoch 51: val_loss did not improve from 0.99492
46/46 - 0s - loss: 0.8952 - mae: 0.6144 - val_loss: 1.0717 - val_mae: 0.7241 - 349ms/epoch - 8ms/step
Epoch 52/1000

Epoch 52: val_loss did not improve from 0.99492
46/46 - 0s - loss: 0.9127 - mae: 0.6269 - val_loss: 1.0707 - val_mae: 0.7204 - 368ms/epoch - 8ms/step
Epoch 53/1000

Epoch 53: val_loss did not improve from 0.99492
46/46 - 0s - loss: 0.9117 - mae: 0.6258 - val_loss: 1.0561 - val_mae: 0.6990 - 340ms/epoch - 7ms/step
Epoch 54/1000

Epoch 54:

In [13]:
def create_fndnn(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(units=20, input_dim=input_dim, activation='relu'))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dense(units=output_dim, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [16]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_train_test_datasets()
model = create_fndnn(input_dim=X_train[0].shape[1], output_dim=Y_train[0].shape[1])
train_test_models(model, X_train, Y_train, X_test, Y_test, result_path="audio/dnn_results_1")

Training for sample: 1
Epoch 1/1000

Epoch 1: val_loss improved from inf to 1.58865, saving model to fcdnn.hdf5
48/48 - 1s - loss: 2.9068 - mae: 1.2195 - val_loss: 1.5886 - val_mae: 0.9419 - 1s/epoch - 28ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 1.58865 to 1.14085, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.3347 - mae: 0.8723 - val_loss: 1.1409 - val_mae: 0.8077 - 219ms/epoch - 5ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 1.14085 to 1.04357, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.1089 - mae: 0.8021 - val_loss: 1.0436 - val_mae: 0.7752 - 214ms/epoch - 4ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 1.04357 to 1.01147, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0494 - mae: 0.7833 - val_loss: 1.0115 - val_mae: 0.7650 - 236ms/epoch - 5ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 1.01147 to 0.99764, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0268 - mae: 0.7766 - val_loss: 0.9976 - val_mae: 0.7608 - 221ms/epoch - 5ms/step
Epoch 6/

Epoch 43/1000

Epoch 43: val_loss improved from 0.61911 to 0.61652, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.5871 - mae: 0.5402 - val_loss: 0.6165 - val_mae: 0.5511 - 201ms/epoch - 4ms/step
Epoch 44/1000

Epoch 44: val_loss improved from 0.61652 to 0.61346, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.5836 - mae: 0.5375 - val_loss: 0.6135 - val_mae: 0.5508 - 205ms/epoch - 4ms/step
Epoch 45/1000

Epoch 45: val_loss improved from 0.61346 to 0.60940, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.5799 - mae: 0.5368 - val_loss: 0.6094 - val_mae: 0.5478 - 220ms/epoch - 5ms/step
Epoch 46/1000

Epoch 46: val_loss improved from 0.60940 to 0.60717, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.5764 - mae: 0.5343 - val_loss: 0.6072 - val_mae: 0.5436 - 214ms/epoch - 4ms/step
Epoch 47/1000

Epoch 47: val_loss improved from 0.60717 to 0.60525, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.5727 - mae: 0.5315 - val_loss: 0.6052 - val_mae: 0.5450 - 204ms/epoch - 4ms/step
Epoch 48/1000



48/48 - 0s - loss: 0.4922 - mae: 0.4813 - val_loss: 0.5431 - val_mae: 0.4999 - 236ms/epoch - 5ms/step
Epoch 87/1000

Epoch 87: val_loss improved from 0.54314 to 0.54028, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.4904 - mae: 0.4801 - val_loss: 0.5403 - val_mae: 0.4991 - 196ms/epoch - 4ms/step
Epoch 88/1000

Epoch 88: val_loss improved from 0.54028 to 0.53938, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.4889 - mae: 0.4785 - val_loss: 0.5394 - val_mae: 0.4998 - 207ms/epoch - 4ms/step
Epoch 89/1000

Epoch 89: val_loss improved from 0.53938 to 0.53864, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.4878 - mae: 0.4782 - val_loss: 0.5386 - val_mae: 0.4948 - 195ms/epoch - 4ms/step
Epoch 90/1000

Epoch 90: val_loss did not improve from 0.53864
48/48 - 0s - loss: 0.4869 - mae: 0.4770 - val_loss: 0.5387 - val_mae: 0.4956 - 205ms/epoch - 4ms/step
Epoch 91/1000

Epoch 91: val_loss improved from 0.53864 to 0.53584, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.4852 - mae: 0.4761 - val_l

Epoch 131/1000

Epoch 131: val_loss did not improve from 0.51017
48/48 - 0s - loss: 0.4460 - mae: 0.4526 - val_loss: 0.5118 - val_mae: 0.4758 - 204ms/epoch - 4ms/step
Epoch 132/1000

Epoch 132: val_loss improved from 0.51017 to 0.50966, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.4454 - mae: 0.4516 - val_loss: 0.5097 - val_mae: 0.4743 - 212ms/epoch - 4ms/step
Epoch 133/1000

Epoch 133: val_loss did not improve from 0.50966
48/48 - 0s - loss: 0.4452 - mae: 0.4511 - val_loss: 0.5098 - val_mae: 0.4774 - 188ms/epoch - 4ms/step
Epoch 134/1000

Epoch 134: val_loss improved from 0.50966 to 0.50956, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.4443 - mae: 0.4516 - val_loss: 0.5096 - val_mae: 0.4772 - 221ms/epoch - 5ms/step
Epoch 135/1000

Epoch 135: val_loss did not improve from 0.50956
48/48 - 0s - loss: 0.4448 - mae: 0.4514 - val_loss: 0.5102 - val_mae: 0.4769 - 190ms/epoch - 4ms/step
Epoch 136/1000

Epoch 136: val_loss improved from 0.50956 to 0.50849, saving model to fcdnn.hdf5
48/4

48/48 - 0s - loss: 0.4255 - mae: 0.4386 - val_loss: 0.4942 - val_mae: 0.4696 - 174ms/epoch - 4ms/step
Epoch 178/1000

Epoch 178: val_loss did not improve from 0.49405
48/48 - 0s - loss: 0.4251 - mae: 0.4393 - val_loss: 0.4956 - val_mae: 0.4674 - 204ms/epoch - 4ms/step
Epoch 179/1000

Epoch 179: val_loss did not improve from 0.49405
48/48 - 0s - loss: 0.4249 - mae: 0.4390 - val_loss: 0.4959 - val_mae: 0.4680 - 188ms/epoch - 4ms/step
Epoch 180/1000

Epoch 180: val_loss did not improve from 0.49405
48/48 - 0s - loss: 0.4241 - mae: 0.4381 - val_loss: 0.4962 - val_mae: 0.4672 - 189ms/epoch - 4ms/step
Epoch 181/1000

Epoch 181: val_loss improved from 0.49405 to 0.49394, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.4241 - mae: 0.4386 - val_loss: 0.4939 - val_mae: 0.4674 - 205ms/epoch - 4ms/step
Epoch 182/1000

Epoch 182: val_loss did not improve from 0.49394
48/48 - 0s - loss: 0.4236 - mae: 0.4375 - val_loss: 0.4956 - val_mae: 0.4700 - 205ms/epoch - 4ms/step
Epoch 183/1000

Epoch 183: val_

...

46/46 - 0s - loss: 0.2539 - mae: 0.3242 - val_loss: 0.2981 - val_mae: 0.3429 - 161ms/epoch - 3ms/step
Epoch 332/1000

Epoch 332: val_loss did not improve from 0.29722
46/46 - 0s - loss: 0.2543 - mae: 0.3240 - val_loss: 0.2985 - val_mae: 0.3433 - 186ms/epoch - 4ms/step
Epoch 333/1000

Epoch 333: val_loss improved from 0.29722 to 0.29706, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2535 - mae: 0.3237 - val_loss: 0.2971 - val_mae: 0.3421 - 205ms/epoch - 4ms/step
Epoch 334/1000

Epoch 334: val_loss did not improve from 0.29706
46/46 - 0s - loss: 0.2541 - mae: 0.3241 - val_loss: 0.3001 - val_mae: 0.3420 - 173ms/epoch - 4ms/step
Epoch 335/1000

Epoch 335: val_loss did not improve from 0.29706
46/46 - 0s - loss: 0.2534 - mae: 0.3235 - val_loss: 0.2990 - val_mae: 0.3420 - 173ms/epoch - 4ms/step
Epoch 336/1000

Epoch 336: val_loss did not improve from 0.29706
46/46 - 0s - loss: 0.2530 - mae: 0.3232 - val_loss: 0.2984 - val_mae: 0.3440 - 205ms/epoch - 4ms/step
Epoch 337/1000

Epoch 337: val_

Epoch 380/1000

Epoch 380: val_loss did not improve from 0.29626
46/46 - 0s - loss: 0.2505 - mae: 0.3218 - val_loss: 0.2983 - val_mae: 0.3426 - 173ms/epoch - 4ms/step
Epoch 381/1000

Epoch 381: val_loss did not improve from 0.29626
46/46 - 0s - loss: 0.2525 - mae: 0.3231 - val_loss: 0.2986 - val_mae: 0.3410 - 173ms/epoch - 4ms/step
Epoch 382/1000

Epoch 382: val_loss did not improve from 0.29626
46/46 - 0s - loss: 0.2509 - mae: 0.3220 - val_loss: 0.2979 - val_mae: 0.3409 - 167ms/epoch - 4ms/step
Epoch 383/1000

Epoch 383: val_loss did not improve from 0.29626
46/46 - 0s - loss: 0.2504 - mae: 0.3218 - val_loss: 0.2980 - val_mae: 0.3403 - 174ms/epoch - 4ms/step
Epoch 384/1000

Epoch 384: val_loss did not improve from 0.29626
46/46 - 0s - loss: 0.2504 - mae: 0.3214 - val_loss: 0.2964 - val_mae: 0.3413 - 197ms/epoch - 4ms/step
Epoch 385/1000

Epoch 385: val_loss did not improve from 0.29626
46/46 - 0s - loss: 0.2500 - mae: 0.3215 - val_loss: 0.2976 - val_mae: 0.3417 - 157ms/epoch - 3ms/ste

46/46 - 0s - loss: 0.2487 - mae: 0.3207 - val_loss: 0.2967 - val_mae: 0.3417 - 190ms/epoch - 4ms/step
Epoch 429/1000

Epoch 429: val_loss improved from 0.29502 to 0.29450, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2485 - mae: 0.3208 - val_loss: 0.2945 - val_mae: 0.3400 - 204ms/epoch - 4ms/step
Epoch 430/1000

Epoch 430: val_loss improved from 0.29450 to 0.29449, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2480 - mae: 0.3206 - val_loss: 0.2945 - val_mae: 0.3393 - 203ms/epoch - 4ms/step
Epoch 431/1000

Epoch 431: val_loss did not improve from 0.29449
46/46 - 0s - loss: 0.2479 - mae: 0.3205 - val_loss: 0.2952 - val_mae: 0.3405 - 192ms/epoch - 4ms/step
Epoch 432/1000

Epoch 432: val_loss did not improve from 0.29449
46/46 - 0s - loss: 0.2480 - mae: 0.3203 - val_loss: 0.2948 - val_mae: 0.3407 - 163ms/epoch - 4ms/step
Epoch 433/1000

Epoch 433: val_loss did not improve from 0.29449
46/46 - 0s - loss: 0.2479 - mae: 0.3205 - val_loss: 0.2961 - val_mae: 0.3416 - 168ms/epoch - 4ms/step

Epoch 477/1000

Epoch 477: val_loss did not improve from 0.29403
46/46 - 0s - loss: 0.2477 - mae: 0.3202 - val_loss: 0.2971 - val_mae: 0.3407 - 168ms/epoch - 4ms/step
Epoch 478/1000

Epoch 478: val_loss did not improve from 0.29403
46/46 - 0s - loss: 0.2477 - mae: 0.3199 - val_loss: 0.2942 - val_mae: 0.3416 - 166ms/epoch - 4ms/step
Epoch 479/1000

Epoch 479: val_loss did not improve from 0.29403
46/46 - 0s - loss: 0.2473 - mae: 0.3198 - val_loss: 0.2945 - val_mae: 0.3401 - 158ms/epoch - 3ms/step
Epoch 480/1000

Epoch 480: val_loss did not improve from 0.29403
46/46 - 0s - loss: 0.2488 - mae: 0.3208 - val_loss: 0.2956 - val_mae: 0.3401 - 177ms/epoch - 4ms/step
Epoch 481/1000

Epoch 481: val_loss improved from 0.29403 to 0.29350, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2468 - mae: 0.3197 - val_loss: 0.2935 - val_mae: 0.3403 - 185ms/epoch - 4ms/step
Epoch 482/1000

Epoch 482: val_loss did not improve from 0.29350
46/46 - 0s - loss: 0.2468 - mae: 0.3195 - val_loss: 0.2944 - val_mae

Epoch 526/1000

Epoch 526: val_loss did not improve from 0.29338
46/46 - 0s - loss: 0.2472 - mae: 0.3192 - val_loss: 0.2942 - val_mae: 0.3414 - 204ms/epoch - 4ms/step
Epoch 527/1000

Epoch 527: val_loss did not improve from 0.29338
46/46 - 0s - loss: 0.2455 - mae: 0.3191 - val_loss: 0.2941 - val_mae: 0.3398 - 173ms/epoch - 4ms/step
Epoch 528/1000

Epoch 528: val_loss did not improve from 0.29338
46/46 - 0s - loss: 0.2459 - mae: 0.3190 - val_loss: 0.2959 - val_mae: 0.3410 - 189ms/epoch - 4ms/step
Epoch 529/1000

Epoch 529: val_loss did not improve from 0.29338
46/46 - 0s - loss: 0.2453 - mae: 0.3191 - val_loss: 0.2952 - val_mae: 0.3398 - 161ms/epoch - 4ms/step
Epoch 530/1000

Epoch 530: val_loss did not improve from 0.29338
46/46 - 0s - loss: 0.2463 - mae: 0.3190 - val_loss: 0.2948 - val_mae: 0.3410 - 173ms/epoch - 4ms/step
Epoch 531/1000

Epoch 531: val_loss did not improve from 0.29338
46/46 - 0s - loss: 0.2483 - mae: 0.3206 - val_loss: 0.2951 - val_mae: 0.3396 - 203ms/epoch - 4ms/ste

In [17]:
def create_fndnn(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(units=40, input_dim=input_dim, activation='relu'))
    model.add(Dense(units=30, activation='relu'))
    model.add(Dense(units=20, activation='relu'))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dense(units=output_dim, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [18]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_train_test_datasets()
model = create_fndnn(input_dim=X_train[0].shape[1], output_dim=Y_train[0].shape[1])
train_test_models(model, X_train, Y_train, X_test, Y_test, result_path="audio/dnn_results_2")

Training for sample: 1
Epoch 1/1000

Epoch 1: val_loss improved from inf to 0.97419, saving model to fcdnn.hdf5
48/48 - 1s - loss: 1.1482 - mae: 0.8219 - val_loss: 0.9742 - val_mae: 0.7611 - 911ms/epoch - 19ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 0.97419 to 0.96532, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0065 - mae: 0.7706 - val_loss: 0.9653 - val_mae: 0.7582 - 312ms/epoch - 6ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 0.96532 to 0.95945, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0008 - mae: 0.7691 - val_loss: 0.9594 - val_mae: 0.7558 - 324ms/epoch - 7ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 0.95945 to 0.92992, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9863 - mae: 0.7574 - val_loss: 0.9299 - val_mae: 0.7263 - 290ms/epoch - 6ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 0.92992 to 0.85565, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9298 - mae: 0.7078 - val_loss: 0.8556 - val_mae: 0.6696 - 280ms/epoch - 6ms/step
Epoch

Epoch 44/1000

Epoch 44: val_loss did not improve from 0.41739
48/48 - 0s - loss: 0.3163 - mae: 0.3766 - val_loss: 0.4228 - val_mae: 0.4300 - 245ms/epoch - 5ms/step
Epoch 45/1000

Epoch 45: val_loss improved from 0.41739 to 0.41311, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3137 - mae: 0.3755 - val_loss: 0.4131 - val_mae: 0.4258 - 291ms/epoch - 6ms/step
Epoch 46/1000

Epoch 46: val_loss improved from 0.41311 to 0.41241, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3101 - mae: 0.3732 - val_loss: 0.4124 - val_mae: 0.4240 - 287ms/epoch - 6ms/step
Epoch 47/1000

Epoch 47: val_loss improved from 0.41241 to 0.41226, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3093 - mae: 0.3721 - val_loss: 0.4123 - val_mae: 0.4240 - 312ms/epoch - 7ms/step
Epoch 48/1000

Epoch 48: val_loss improved from 0.41226 to 0.40942, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3072 - mae: 0.3708 - val_loss: 0.4094 - val_mae: 0.4212 - 291ms/epoch - 6ms/step
Epoch 49/1000

Epoch 49: val_loss improved from

Epoch 91/1000

Epoch 91: val_loss did not improve from 0.38311
48/48 - 0s - loss: 0.2567 - mae: 0.3351 - val_loss: 0.3878 - val_mae: 0.4020 - 250ms/epoch - 5ms/step
Epoch 92/1000

Epoch 92: val_loss did not improve from 0.38311
48/48 - 0s - loss: 0.2558 - mae: 0.3352 - val_loss: 0.3878 - val_mae: 0.4052 - 230ms/epoch - 5ms/step
Epoch 93/1000

Epoch 93: val_loss did not improve from 0.38311
48/48 - 0s - loss: 0.2550 - mae: 0.3345 - val_loss: 0.3851 - val_mae: 0.4020 - 258ms/epoch - 5ms/step
Epoch 94/1000

Epoch 94: val_loss did not improve from 0.38311
48/48 - 0s - loss: 0.2538 - mae: 0.3334 - val_loss: 0.3846 - val_mae: 0.4018 - 242ms/epoch - 5ms/step
Epoch 95/1000

Epoch 95: val_loss did not improve from 0.38311
48/48 - 0s - loss: 0.2550 - mae: 0.3346 - val_loss: 0.3849 - val_mae: 0.4010 - 246ms/epoch - 5ms/step
Epoch 96/1000

Epoch 96: val_loss did not improve from 0.38311
48/48 - 0s - loss: 0.2531 - mae: 0.3333 - val_loss: 0.3870 - val_mae: 0.3973 - 234ms/epoch - 5ms/step
Epoch 97/1

48/48 - 0s - loss: 0.2347 - mae: 0.3205 - val_loss: 0.3709 - val_mae: 0.3924 - 283ms/epoch - 6ms/step
Epoch 138/1000

Epoch 138: val_loss did not improve from 0.37092
48/48 - 0s - loss: 0.2336 - mae: 0.3194 - val_loss: 0.3792 - val_mae: 0.3927 - 232ms/epoch - 5ms/step
Epoch 139/1000

Epoch 139: val_loss did not improve from 0.37092
48/48 - 0s - loss: 0.2321 - mae: 0.3182 - val_loss: 0.3711 - val_mae: 0.3932 - 248ms/epoch - 5ms/step
Epoch 140/1000

Epoch 140: val_loss did not improve from 0.37092
48/48 - 0s - loss: 0.2320 - mae: 0.3184 - val_loss: 0.3751 - val_mae: 0.3966 - 241ms/epoch - 5ms/step
Epoch 141/1000

Epoch 141: val_loss improved from 0.37092 to 0.36860, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.2325 - mae: 0.3189 - val_loss: 0.3686 - val_mae: 0.3900 - 277ms/epoch - 6ms/step
Epoch 142/1000

Epoch 142: val_loss did not improve from 0.36860
48/48 - 0s - loss: 0.2313 - mae: 0.3177 - val_loss: 0.3711 - val_mae: 0.3925 - 233ms/epoch - 5ms/step
Epoch 143/1000

Epoch 143: val_

Epoch 186/1000

Epoch 186: val_loss did not improve from 0.36459
48/48 - 0s - loss: 0.2164 - mae: 0.3089 - val_loss: 0.3732 - val_mae: 0.3897 - 225ms/epoch - 5ms/step
Epoch 187/1000

Epoch 187: val_loss did not improve from 0.36459
48/48 - 0s - loss: 0.2169 - mae: 0.3088 - val_loss: 0.3653 - val_mae: 0.3860 - 233ms/epoch - 5ms/step
Epoch 188/1000

Epoch 188: val_loss did not improve from 0.36459
48/48 - 0s - loss: 0.2160 - mae: 0.3082 - val_loss: 0.3672 - val_mae: 0.3900 - 247ms/epoch - 5ms/step
Epoch 189/1000

Epoch 189: val_loss did not improve from 0.36459
48/48 - 0s - loss: 0.2157 - mae: 0.3087 - val_loss: 0.3679 - val_mae: 0.3864 - 240ms/epoch - 5ms/step
Epoch 190/1000

Epoch 190: val_loss did not improve from 0.36459
48/48 - 0s - loss: 0.2147 - mae: 0.3069 - val_loss: 0.3682 - val_mae: 0.3904 - 230ms/epoch - 5ms/step
Epoch 191/1000

Epoch 191: val_loss did not improve from 0.36459
48/48 - 0s - loss: 0.2156 - mae: 0.3080 - val_loss: 0.3789 - val_mae: 0.4001 - 234ms/epoch - 5ms/ste

...

Epoch 313/1000

Epoch 313: val_loss did not improve from 0.31573
46/46 - 0s - loss: 0.2564 - mae: 0.3208 - val_loss: 0.3176 - val_mae: 0.3520 - 301ms/epoch - 7ms/step
Epoch 314/1000

Epoch 314: val_loss improved from 0.31573 to 0.31525, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.2553 - mae: 0.3199 - val_loss: 0.3153 - val_mae: 0.3540 - 340ms/epoch - 7ms/step
Epoch 315/1000

Epoch 315: val_loss did not improve from 0.31525
46/46 - 0s - loss: 0.2555 - mae: 0.3204 - val_loss: 0.3162 - val_mae: 0.3534 - 276ms/epoch - 6ms/step
Epoch 316/1000

Epoch 316: val_loss did not improve from 0.31525
46/46 - 0s - loss: 0.2555 - mae: 0.3208 - val_loss: 0.3174 - val_mae: 0.3521 - 248ms/epoch - 5ms/step
Epoch 317/1000

Epoch 317: val_loss did not improve from 0.31525
46/46 - 0s - loss: 0.2546 - mae: 0.3195 - val_loss: 0.3155 - val_mae: 0.3508 - 263ms/epoch - 6ms/step
Epoch 318/1000

Epoch 318: val_loss did not improve from 0.31525
46/46 - 0s - loss: 0.2545 - mae: 0.3196 - val_loss: 0.3182 - val_mae

46/46 - 0s - loss: 0.2489 - mae: 0.3155 - val_loss: 0.3157 - val_mae: 0.3508 - 304ms/epoch - 7ms/step
Epoch 362/1000

Epoch 362: val_loss did not improve from 0.31403
46/46 - 0s - loss: 0.2496 - mae: 0.3165 - val_loss: 0.3171 - val_mae: 0.3512 - 331ms/epoch - 7ms/step
Epoch 363/1000

Epoch 363: val_loss did not improve from 0.31403
46/46 - 0s - loss: 0.2484 - mae: 0.3155 - val_loss: 0.3175 - val_mae: 0.3548 - 275ms/epoch - 6ms/step
Epoch 364/1000

Epoch 364: val_loss did not improve from 0.31403
46/46 - 0s - loss: 0.2490 - mae: 0.3162 - val_loss: 0.3177 - val_mae: 0.3490 - 302ms/epoch - 7ms/step
Epoch 365/1000

Epoch 365: val_loss did not improve from 0.31403
46/46 - 0s - loss: 0.2485 - mae: 0.3152 - val_loss: 0.3179 - val_mae: 0.3505 - 370ms/epoch - 8ms/step
Epoch 366/1000

Epoch 366: val_loss did not improve from 0.31403
46/46 - 0s - loss: 0.2485 - mae: 0.3151 - val_loss: 0.3175 - val_mae: 0.3510 - 285ms/epoch - 6ms/step
Epoch 367/1000

Epoch 367: val_loss did not improve from 0.3140


Epoch 410: val_loss did not improve from 0.31188
46/46 - 0s - loss: 0.2454 - mae: 0.3131 - val_loss: 0.3148 - val_mae: 0.3485 - 319ms/epoch - 7ms/step
Epoch 411/1000

Epoch 411: val_loss did not improve from 0.31188
46/46 - 0s - loss: 0.2440 - mae: 0.3125 - val_loss: 0.3139 - val_mae: 0.3483 - 351ms/epoch - 8ms/step
Epoch 412/1000

Epoch 412: val_loss did not improve from 0.31188
46/46 - 0s - loss: 0.2435 - mae: 0.3126 - val_loss: 0.3152 - val_mae: 0.3488 - 302ms/epoch - 7ms/step
Epoch 413/1000

Epoch 413: val_loss did not improve from 0.31188
46/46 - 0s - loss: 0.2440 - mae: 0.3129 - val_loss: 0.3127 - val_mae: 0.3481 - 237ms/epoch - 5ms/step
Epoch 414/1000

Epoch 414: val_loss did not improve from 0.31188
46/46 - 0s - loss: 0.2435 - mae: 0.3122 - val_loss: 0.3145 - val_mae: 0.3474 - 296ms/epoch - 6ms/step
Epoch 415/1000

Epoch 415: val_loss did not improve from 0.31188
46/46 - 0s - loss: 0.2437 - mae: 0.3124 - val_loss: 0.3154 - val_mae: 0.3482 - 254ms/epoch - 6ms/step
Epoch 416/100

46/46 - 0s - loss: 0.2423 - mae: 0.3119 - val_loss: 0.3091 - val_mae: 0.3471 - 223ms/epoch - 5ms/step
Epoch 459/1000

Epoch 459: val_loss did not improve from 0.30796
46/46 - 0s - loss: 0.2409 - mae: 0.3110 - val_loss: 0.3108 - val_mae: 0.3475 - 250ms/epoch - 5ms/step
Epoch 460/1000

Epoch 460: val_loss did not improve from 0.30796
46/46 - 0s - loss: 0.2413 - mae: 0.3110 - val_loss: 0.3095 - val_mae: 0.3482 - 248ms/epoch - 5ms/step
Epoch 461/1000

Epoch 461: val_loss did not improve from 0.30796
46/46 - 0s - loss: 0.2408 - mae: 0.3105 - val_loss: 0.3117 - val_mae: 0.3467 - 301ms/epoch - 7ms/step
Epoch 462/1000

Epoch 462: val_loss did not improve from 0.30796
46/46 - 0s - loss: 0.2408 - mae: 0.3105 - val_loss: 0.3116 - val_mae: 0.3463 - 310ms/epoch - 7ms/step
Epoch 463/1000

Epoch 463: val_loss did not improve from 0.30796
46/46 - 0s - loss: 0.2406 - mae: 0.3099 - val_loss: 0.3102 - val_mae: 0.3516 - 304ms/epoch - 7ms/step
Epoch 464/1000

Epoch 464: val_loss did not improve from 0.3079

46/46 - 0s - loss: 0.2400 - mae: 0.3101 - val_loss: 0.3106 - val_mae: 0.3464 - 224ms/epoch - 5ms/step
Epoch 508/1000

Epoch 508: val_loss did not improve from 0.30735
46/46 - 0s - loss: 0.2406 - mae: 0.3109 - val_loss: 0.3140 - val_mae: 0.3474 - 230ms/epoch - 5ms/step
Epoch 509/1000

Epoch 509: val_loss did not improve from 0.30735
46/46 - 0s - loss: 0.2400 - mae: 0.3104 - val_loss: 0.3126 - val_mae: 0.3468 - 231ms/epoch - 5ms/step
Epoch 510/1000

Epoch 510: val_loss did not improve from 0.30735
46/46 - 0s - loss: 0.2387 - mae: 0.3089 - val_loss: 0.3116 - val_mae: 0.3481 - 276ms/epoch - 6ms/step
Epoch 511/1000

Epoch 511: val_loss did not improve from 0.30735
46/46 - 0s - loss: 0.2420 - mae: 0.3115 - val_loss: 0.3096 - val_mae: 0.3460 - 290ms/epoch - 6ms/step
Epoch 512/1000

Epoch 512: val_loss did not improve from 0.30735
46/46 - 0s - loss: 0.2397 - mae: 0.3100 - val_loss: 0.3078 - val_mae: 0.3474 - 263ms/epoch - 6ms/step
Epoch 513/1000

Epoch 513: val_loss did not improve from 0.3073